In [8]:
from config import Config
from modules.trainer import train
from modules.model import MyModel
from modules.dataset import MyDataSet
from modules.dataloader import collate_fn_no_label

import sys, os
import torch
import pandas as pd
from tqdm.auto import tqdm
from torch.utils.data import DataLoader
from transformers import AutoFeatureExtractor


    

In [6]:
import sys,os
sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname("/scratch/network/mk8574/audio_sentiment_challenge/baseline_youhan/modules/"))))

In [7]:
import sys,os
sys.path

['/scratch/network/mk8574/audio_sentiment_challenge/baseline_dy/xgboost',
 '/home/mk8574/.conda/envs/mk8574_3.10/lib/python310.zip',
 '/home/mk8574/.conda/envs/mk8574_3.10/lib/python3.10',
 '/home/mk8574/.conda/envs/mk8574_3.10/lib/python3.10/lib-dynload',
 '',
 '/home/mk8574/.conda/envs/mk8574_3.10/lib/python3.10/site-packages',
 '/scratch/network/mk8574/audio_sentiment_challenge',
 '/scratch/network/mk8574/audio_sentiment_challenge/baseline_youhan']

In [10]:
from torch import Tensor
import numpy as np
from tqdm import tqdm
def softmax(x):
    y = np.exp(x)
    f_x = y / np.sum(np.exp(x))
    return f_x
config = Config()
device = torch.device(f'cuda:{config.device}') if torch.cuda.is_available() else torch.device('cpu')
arg = "20231124_064827"
working_path = os.path.join('/scratch/network/mk8574/audio_sentiment_challenge/baseline_youhan/results', arg)
model = MyModel(working_path, mode='test').to(device)

test_df = pd.read_csv(os.path.join("/scratch/network/mk8574/audio_sentiment_challenge/baseline_dy/xgboost/valid.csv"))
test_df.reset_index(drop=True, inplace=True)

feature_extractor = AutoFeatureExtractor.from_pretrained(config.pretrained_name)
test_dataset = MyDataSet(test_df, feature_extractor, mode='test', data_path=config.data_path)

test_loader = DataLoader(test_dataset, batch_size=config.batch_size, shuffle=False, collate_fn=collate_fn_no_label, num_workers=config.num_workers)

model.eval()
preds = []
pred_label = [[] for _ in range(6)]
with torch.no_grad():
    for x in tqdm(iter(test_loader)):
        x = x.to(device)
        output = model(x)
        preds += output.argmax(-1).detach().cpu().numpy().tolist()
        for i in range(len(output)):

            output[i] = Tensor(softmax(output[i].cpu().numpy().tolist()))


        for i in range(6):
            pred_label[i]+=output[:,i].detach().cpu().numpy().tolist()
# submission = pd.read_csv(os.path.join(config.data_path, 'sample_submission.csv'))
# submission['label'] = preds
# for i in range(6):
#     submission["prob_"+str(i)] = pred_label[i]
# submission.to_csv(os.path.join(working_path, 'prob_val2.csv'), index=False)

  0%|          | 0/26 [00:00<?, ?it/s]2023-11-25 05:50:58.071990: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-25 05:50:58.071990: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-25 05:50:58.071990: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-25 05:50:58.080962: I tensorflow/core/util/port.cc:113] oneDNN custom op

In [11]:
submission = pd.read_csv(os.path.join("/scratch/network/mk8574/audio_sentiment_challenge/baseline_dy/xgboost/valid.csv"))
submission['label'] = preds
for i in range(6):
    submission["prob_"+str(i)] = pred_label[i]
submission.to_csv(os.path.join("/scratch/network/mk8574/audio_sentiment_challenge/baseline_dy/xgboost", 'prob_val2.csv'), index=False)

In [2]:
a= pd.read_csv("/scratch/network/mk8574/audio_sentiment_challenge/baseline_dy/xgboost/prob_val.csv")
b = pd.read_csv("/scratch/network/mk8574/audio_sentiment_challenge/baseline_dy/xgboost/validans.csv")
c= pd.read_csv("/scratch/network/mk8574/audio_sentiment_challenge/baseline_dy/xgboost/prob_val2.csv")
x,y=0,0
for i in range(len(a)):
    if c["label"][i]==a["label"][i]:
        x+=1
    else:
        y+=1
print(x,y)

759 242


In [3]:
new = pd.concat([a,c],axis=1)

In [4]:
new.head()

,id,path,label,prob_0,prob_1,prob_2,prob_3,prob_4,prob_5,id,path,label,prob_0,prob_1,prob_2,prob_3,prob_4,prob_5
0,TRAIN_1501,./train/TRAIN_1501.wav,0,9.999882e-01,2.698073e-08,9.972004e-13,7.481026e-10,2.284406e-11,1.174704e-05,TRAIN_1501,./train/TRAIN_1501.wav,5,0.114615,0.004971,0.000283,0.007948,0.000094,0.872090
1,TRAIN_2586,./train/TRAIN_2586.wav,5,3.569975e-07,1.437433e-06,5.319619e-14,2.680158e-10,3.765693e-11,9.999982e-01,TRAIN_2586,./train/TRAIN_2586.wav,5,0.001258,0.001267,0.000135,0.000204,0.000346,0.996790
2,TRAIN_2653,./train/TRAIN_2653.wav,4,2.561158e-09,3.974795e-09,1.147402e-09,3.107179e-10,1.000000e+00,1.520057e-09,TRAIN_2653,./train/TRAIN_2653.wav,4,0.006031,0.003878,0.012162,0.002638,0.973926,0.001364
3,TRAIN_1055,./train/TRAIN_1055.wav,4,4.970420e-04,3.404393e-10,3.311203e-13,1.407155e-10,9.452709e-01,5.423207e-02,TRAIN_1055,./train/TRAIN_1055.wav,4,0.035601,0.001977,0.003281,0.001916,0.500885,0.456340
4,TRAIN_0705,./train/TRAIN_0705.wav,2,6.097657e-10,2.213600e-02,9.764557e-01,1.408232e-03,1.648795e-09,1.984731e-10,TRAIN_0705,./train/TRAIN_0705.wav,1,0.001558,0.466801,0.295990,0.234605,0.000560,0.000486


In [5]:
new=new.drop(["label","id","path"],axis=1)

In [1]:
import numpy as np
import pandas as pd
import os
import librosa
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score
from scipy.stats import skew
SAMPLE_RATE = 16000

In [59]:
xgb = XGBClassifier(max_depth=5, learning_rate=0.05, n_estimators=3000,
                    n_jobs=-1, random_state=0, reg_alpha=0.2, 
                    colsample_bylevel=0.9, colsample_bytree=0.9)

In [6]:
new.head()

,prob_0,prob_1,prob_2,prob_3,prob_4,prob_5,prob_0,prob_1,prob_2,prob_3,prob_4,prob_5
0,9.999882e-01,2.698073e-08,9.972004e-13,7.481026e-10,2.284406e-11,1.174704e-05,0.114615,0.004971,0.000283,0.007948,0.000094,0.872090
1,3.569975e-07,1.437433e-06,5.319619e-14,2.680158e-10,3.765693e-11,9.999982e-01,0.001258,0.001267,0.000135,0.000204,0.000346,0.996790
2,2.561158e-09,3.974795e-09,1.147402e-09,3.107179e-10,1.000000e+00,1.520057e-09,0.006031,0.003878,0.012162,0.002638,0.973926,0.001364
3,4.970420e-04,3.404393e-10,3.311203e-13,1.407155e-10,9.452709e-01,5.423207e-02,0.035601,0.001977,0.003281,0.001916,0.500885,0.456340
4,6.097657e-10,2.213600e-02,9.764557e-01,1.408232e-03,1.648795e-09,1.984731e-10,0.001558,0.466801,0.295990,0.234605,0.000560,0.000486


In [7]:
cols=pd.Series(new.columns)
for dup in new.columns[new.columns.duplicated(keep=False)]: 
    cols[new.columns.get_loc(dup)] = ([dup + '.' + str(d_idx) 
                                     if d_idx != 0 
                                     else dup 
                                     for d_idx in range(new.columns.get_loc(dup).sum())]
                                    )
new.columns=cols

In [8]:
new.head()

,prob_0,prob_1,prob_2,prob_3,prob_4,prob_5,prob_0.1,prob_1.1,prob_2.1,prob_3.1,prob_4.1,prob_5.1
0,9.999882e-01,2.698073e-08,9.972004e-13,7.481026e-10,2.284406e-11,1.174704e-05,0.114615,0.004971,0.000283,0.007948,0.000094,0.872090
1,3.569975e-07,1.437433e-06,5.319619e-14,2.680158e-10,3.765693e-11,9.999982e-01,0.001258,0.001267,0.000135,0.000204,0.000346,0.996790
2,2.561158e-09,3.974795e-09,1.147402e-09,3.107179e-10,1.000000e+00,1.520057e-09,0.006031,0.003878,0.012162,0.002638,0.973926,0.001364
3,4.970420e-04,3.404393e-10,3.311203e-13,1.407155e-10,9.452709e-01,5.423207e-02,0.035601,0.001977,0.003281,0.001916,0.500885,0.456340
4,6.097657e-10,2.213600e-02,9.764557e-01,1.408232e-03,1.648795e-09,1.984731e-10,0.001558,0.466801,0.295990,0.234605,0.000560,0.000486


In [9]:
new.to_csv(os.path.join("/scratch/network/mk8574/audio_sentiment_challenge/baseline_dy/xgboost", 'prob_plus.csv'), index=False)

In [63]:
xgb.fit(new,b["label"])

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=0.9, colsample_bynode=None,
              colsample_bytree=0.9, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.05, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=3000, n_jobs=-1,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [ ]:
val2,c, 나중꺼가가 유한씨꺼

In [51]:
a= pd.read_csv("/scratch/network/mk8574/audio_sentiment_challenge/baseline_dy/xgboost/prob_whisper.csv")
c= pd.read_csv("/scratch/network/mk8574/audio_sentiment_challenge/baseline_dy/xgboost/prob.csv")

In [52]:
newd = pd.concat([a,c],axis=1)
newd.head()


,id,label,prob_0,prob_1,prob_2,prob_3,prob_4,prob_5,id,label,prob_0,prob_1,prob_2,prob_3,prob_4,prob_5
0,TEST_0000,5,7.514885e-10,3.069834e-10,2.105965e-13,9.609169e-12,1.328578e-09,1.000000e+00,TEST_0000,5,0.001956,0.000509,0.000432,0.000272,0.005476,0.991356
1,TEST_0001,1,3.004239e-01,6.904778e-01,8.723975e-07,1.912085e-10,1.230165e-06,9.096202e-03,TEST_0001,0,0.520072,0.277725,0.007695,0.054015,0.001753,0.138740
2,TEST_0002,2,8.448483e-09,1.109684e-10,9.999399e-01,5.891908e-05,1.125340e-06,1.120816e-12,TEST_0002,2,0.000258,0.014070,0.969636,0.001426,0.014425,0.000184
3,TEST_0003,3,2.985870e-08,2.753863e-09,7.746400e-06,9.999916e-01,1.748746e-09,6.455159e-07,TEST_0003,5,0.003342,0.000479,0.000322,0.000815,0.000856,0.994186
4,TEST_0004,0,9.943925e-01,1.071751e-09,1.189908e-12,9.201906e-10,5.529615e-03,7.791320e-05,TEST_0004,0,0.974327,0.000405,0.000859,0.004143,0.019196,0.001071


In [53]:
newd=newd.drop(["label","id"],axis=1)
newd.head()

,prob_0,prob_1,prob_2,prob_3,prob_4,prob_5,prob_0,prob_1,prob_2,prob_3,prob_4,prob_5
0,7.514885e-10,3.069834e-10,2.105965e-13,9.609169e-12,1.328578e-09,1.000000e+00,0.001956,0.000509,0.000432,0.000272,0.005476,0.991356
1,3.004239e-01,6.904778e-01,8.723975e-07,1.912085e-10,1.230165e-06,9.096202e-03,0.520072,0.277725,0.007695,0.054015,0.001753,0.138740
2,8.448483e-09,1.109684e-10,9.999399e-01,5.891908e-05,1.125340e-06,1.120816e-12,0.000258,0.014070,0.969636,0.001426,0.014425,0.000184
3,2.985870e-08,2.753863e-09,7.746400e-06,9.999916e-01,1.748746e-09,6.455159e-07,0.003342,0.000479,0.000322,0.000815,0.000856,0.994186
4,9.943925e-01,1.071751e-09,1.189908e-12,9.201906e-10,5.529615e-03,7.791320e-05,0.974327,0.000405,0.000859,0.004143,0.019196,0.001071


In [54]:
cols=pd.Series(new.columns)
for dup in newd.columns[newd.columns.duplicated(keep=False)]: 
    cols[newd.columns.get_loc(dup)] = ([dup + '.' + str(d_idx) 
                                     if d_idx != 0 
                                     else dup 
                                     for d_idx in range(newd.columns.get_loc(dup).sum())]
                                    )
newd.columns=cols
newd.head()

,prob_0,prob_1,prob_2,prob_3,prob_4,prob_5,prob_0.1,prob_1.1,prob_2.1,prob_3.1,prob_4.1,prob_5.1
0,7.514885e-10,3.069834e-10,2.105965e-13,9.609169e-12,1.328578e-09,1.000000e+00,0.001956,0.000509,0.000432,0.000272,0.005476,0.991356
1,3.004239e-01,6.904778e-01,8.723975e-07,1.912085e-10,1.230165e-06,9.096202e-03,0.520072,0.277725,0.007695,0.054015,0.001753,0.138740
2,8.448483e-09,1.109684e-10,9.999399e-01,5.891908e-05,1.125340e-06,1.120816e-12,0.000258,0.014070,0.969636,0.001426,0.014425,0.000184
3,2.985870e-08,2.753863e-09,7.746400e-06,9.999916e-01,1.748746e-09,6.455159e-07,0.003342,0.000479,0.000322,0.000815,0.000856,0.994186
4,9.943925e-01,1.071751e-09,1.189908e-12,9.201906e-10,5.529615e-03,7.791320e-05,0.974327,0.000405,0.000859,0.004143,0.019196,0.001071


In [64]:
y_preds = xgb.predict(newd) #검증

In [65]:
a= pd.read_csv("/scratch/network/mk8574/audio_sentiment_challenge/baseline_dy/xgboost/submission.csv")
a["label"]=y_preds
a.to_csv(os.path.join("/scratch/network/mk8574/audio_sentiment_challenge/baseline_dy/xgboost", 'stacked.csv'), index=False)

In [ ]:
x,y=0,0
for i in range(len(a)):
    if c["label"][i]==a["label"][i]:
        x+=1
    else:
        y+=1
print(x,y)